# RTR Connectivity Graph

- Index pruned trends to pruned keywords
- Build graph

## Next steps:

- Make Network With Pruned Trends

Create new structure with: 
- keywords from given abstract
- trending words from given abstract

TODO: include pubmed link to original abstract

In [225]:
# SETUP
import operator
import pickle
def setup():
    %run rtr_bow.ipynb

In [240]:
def unpickle_data():
    with open("trends.pickle", "rb") as fp:
        trends = pickle.load(fp)
    with open("records.pickle", "rb") as fp:
        records = pickle.load(fp)
    with open("df.pickle", "rb") as fp:
        df = pickle.load(fp)
    return trends, records, df

Create data structure to hold trend and associated keywords

In [241]:
from collections import Counter
class Trend:
    def __init__(self, trend):
        self.trend = trend
        #using a set doesn't allow duplicates
        #self.keywords = set()
        self.keywords = Counter()
    def add_keyword(self, keyword):
        #self.keywords.add(keyword)
        self.keywords.update({keyword:1})
    def as_dict(self):
        return {self.trend:self.keywords}

In [242]:
def pair_trends_keywords(df, window_records, n_trends, n_keywords, from_sql = False):
    df_pruned = df.iloc[:n_trends,:]
    trends_list = []
    for entry in df_pruned.key:
        trend_str = ' '.join(entry)
        trend = Trend(trend_str)
        trends_list.append(trend)
    # get filtered keywords
    top_keywords = df.iloc[:n_keywords,0]
    print(top_keywords)
    top_keywords_text = [keyword[0] for keyword in top_keywords]
    #print(top_keywords_text)
    # TRIPLE LOOP - sure to be a bottleneck
    for trend in trends_list:
        for window in window_records:
            # direct method only --
            # update to include sql compatibility
            # when sql is working
            for abstract_record in window:
                if from_sql:
                    abstract = abstract_record[0]
                else:
                    abstract = abstract_record['Abstract'][0]
                if trend.trend in abstract:
                    if from_sql:
                        keywords = str.split(abstract_record[2], ',')
                    else:
                        keywords = abstract_record['Keywords']
                    for keyword in keywords:
                        if keyword in top_keywords_text:
                            #print("matching trend", trend.trend, "and keyword", keyword)
                            trend.add_keyword(keyword)
                #else:
                    #print("not in abstract")
    return trends_list

In [243]:
def print_trend_dict(trends_list, N=50):
    n = 0
    for trend in trends_list:
        print(trend.as_dict())
        n += 1
        if n > N:
            return

## Building the graph

In [244]:
import networkx as nx
import operator

In [245]:
def populate_graph(trends_list):
    G = nx.Graph()
    for trend in trends_list:
        node1 = trend.trend
        for key, item in trend.keywords.items():
            node2 = key
            if G.has_edge(node1, node2):
                G[node1][node2]['weight'] += item
            else:
                # new edge. add with weight=1
                G.add_node(node1, is_key = False)
                G.add_node(node2, is_key = True)
                G.add_edge(node1, node2, weight=item)
    for u, v, d in G.edges(data=True):
        weight = d['weight']
    return G

## Find ten most heavily weighted edges of the graph

In [246]:
def top_N_trends(G, keyword, n=10):
    node_edges = G.edges(keyword.lower())
    edges_dict = {}
    for edge in node_edges:
        key = edge[1]
        value = G[edge[0]][edge[1]]['weight']
        edges_dict.update({key:value})
    index = 1
    trends = []
    for key, value in sorted(edges_dict.items(), key=operator.itemgetter(1), reverse=True):
        trends.append(' '.join([key, str(value)]))
        index += 1
        if index > n:
            break
    return trends

## Find which keywords made the graph

In [247]:
def get_hot_keywords(G, N=20):
    keywords =  {}
    for node in G.nodes(data=True):
        if node[1]['is_key']:
            total_wt = 0
            for edge in G.edges(node[0], data=True):
                total_wt += edge[2]['weight']
            keywords.update({node[0]: total_wt})
    n = 0
    hot_keywords = []
    for item in sorted(keywords.items(), key=operator.itemgetter(1), reverse=True):
        n += 1
        if n > N:
            break
        else:
            hot_keywords.append(item)
    return hot_keywords

In [248]:
def convert_trends(trends_list):
    trends_converted = {}
    for key, item in trends.items():
        key_new = ' '.join(key)
        item_new = item['vals']
        trends_converted.update({key_new:item_new})
    return bow_new

In [249]:
# pickle graph
def pickle_graph():
    try: 
        with open("/home/ericbarnhill/Documents/code/insight/rtr/G.txt", "wb") as fp:
            pickle.dump(G, fp)
    except:
        print('error')
        
def unpickle_graph():
    try: 
        with open("/home/ericbarnhill/Documents/code/insight/rtr/G.txt", "rb") as fp:
            G = pickle.load(fp)
    except:
        print('error')
    return G
        
def unpickle_bow_trends():
    try: 
        with open("/home/ericbarnhill/Documents/code/insight/rtr/bow_converted.txt", "rb") as fp:
            bow_trends = pickle.load(fp)
    except:
        print('error')
    return bow_trends

In [250]:
def graph_figure(G):
    nx.draw_networkx_nodes(G[2], nx.spring_layout(G[2]), node_size=10)
    nx.draw_networkx_edges(G[2], nx.spring_layout(G[2]), alpha=0.4)
    plt.xlim((-0.1, 0.1))
    plt.ylim((-0.1, 0.1))
    plt.show()

In [251]:
import operator
def centrality_measures(G):
    dc = nx.degree_centrality(G)
    bc = nx.betweenness_centrality(G)
    ec = nx.eigenvector_centrality_numpy(G)
    dc = sorted(dc.items(), key=operator.itemgetter(1), reverse=True)
    bc = sorted(bc.items(), key=operator.itemgetter(1), reverse=True)
    ec = sorted(ec.items(), key=operator.itemgetter(1), reverse=True)
    return dc, bc, ec
#    for key, value in sorted(bc.items(), key=operator.itemgetter(1), reverse=True):
#        print(key, value)
#        n += 1
#        if n > 10:
#            break

In [252]:
def run_nb(set_up=False):
    if set_up:
        setup()
    N_TRENDS = 1000
    N_KEYWORDS = 3000
    L = 5
    trends, records, df = unpickle_data()
    trends_list = pair_trends_keywords(df, records,
                                       N_TRENDS, N_KEYWORDS, from_sql = True)
    print("Top 20 trends:")
    print_trend_dict(trends_list, 20)
    G = populate_graph(trends_list)
    print("MRI trends:")
    mri_trends = top_N_trends(G, 'magnetic resonance imaging')
    print(mri_trends)
    print("Hottest keywords:")
    hot_keywords = get_hot_keywords(G)
    print(hot_keywords)
    dc, bc, ec = centrality_measures(G)
    print("Top degree centrality:", list(dc)[:L])
    print("Top betweenness centrality:", list(bc)[:L])
    print("Top eigencentrality:", list(ec)[:L])
    trends_converted = convert_trends(trends_list)

In [253]:
run_nb(False)

62               (resonance, imaging, contrast)
291       (prospective, randomized, controlled)
332             (tomography, angiography, octa)
422                    (drug, delivery, system)
426                       (frontal, gyrus, ifg)
638           (adverse, cardiovascular, events)
496                  (serious, adverse, events)
676                (nonalcoholic, fatty, liver)
282          (independent, component, analysis)
577            (systemic, lupus, erythematosus)
649    (fluorodeoxyglucose, positron, emission)
470                 (using, scanning, electron)
398               (resonance, angiography, mra)
543                      (grey, matter, volume)
537                  (using, cox, proportional)
588                (anterior, cerebral, artery)
568        (functional, connectivity, analysis)
125             (death, myocardial, infarction)
61                 (cardiac, death, myocardial)
678         (adolescent, idiopathic, scoliosis)
155        (retrospectively, reviewed, m

NameError: name 'trends' is not defined

In [ ]:
dc, bc, ec = centrality_measures(G)
L = 10
web_list = {'imaging':1, 'tomography':2, 'rats':3, 'ultrasonography':4, \
            'carcinoma':5, 'mice':6, 'echocardiography':7, 'diagnosis':8, \
           'microscopy':9, 'cartilage':10}
dc_list = {}
bc_list = {}
ec_list = {}
for n in range(L):
    dc_list.update({list(dc)[n][0]:n+1})
    bc_list.update({list(bc)[n][0]:n+1})
    ec_list.update({list(ec)[n][0]:n+1})
    


In [ ]:
import pandas as pd
import numpy as np
web_df['listnum'] = np.tile(1, (web_df.shape[0], 1))
dc_df = pd.DataFrame(dc_list, index=[1]).melt()
dc_df['listnum'] = np.tile(2, (web_df.shape[0], 1))
ec_df = pd.DataFrame(ec_list, index=[2]).melt()
ec_df['listnum'] = np.tile(3, (web_df.shape[0], 1))
bc_df = pd.DataFrame(bc_list, index=[3]).melt()
bc_df['listnum'] = np.tile(4, (web_df.shape[0], 1))

In [ ]:
df = pd.concat([web_df, dc_df, ec_df, bc_df])
df = df.reset_index()
df

In [ ]:
import altair as alt
chart = alt.Chart(df, width=400).mark_line().encode(
    x = 'listnum:O',
    y = 'value:O', 
    color = 'variable'
)
chart

In [ ]:
df_wide